In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_nystrom as file

In [2]:
# Args setup
ntrial = 50
n = 10000
ntest = 100
d = 5
noise = 0.5
features = 1000
samples = 1000
calibrate = [False, True, True]
sample_split = [False, False, True]
marginal_loss = False
retrain_hypers = False
kernel = "gaussian"
approx = "mercer"

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="128GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="12:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
    for i in range(len(calibrate)):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                              calibrate = calibrate[i],
                              sample_split = sample_split[i],
                              marginal_loss = marginal_loss,
                              retrain_hypers = retrain_hypers,
                              features = features, samples = samples,
                              approx = approx,
                              kernel = "gaussian")
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-0787740202121e4e84e2879af53c256a>,
 <Future: pending, key: main-409eeaf87cd6c671623b0672de02f265>,
 <Future: pending, key: main-33893e7566e7a51f530587444ba7313a>,
 <Future: pending, key: main-b1ff4c95dff2c0c26a3170860c12005a>,
 <Future: pending, key: main-01c79944e2470c8d45ac090188c9650c>,
 <Future: pending, key: main-50eca745705bb15f0ac9dd91060b3e38>,
 <Future: pending, key: main-14d7f82fd3640c849ac24b1418503188>,
 <Future: pending, key: main-c011ac003f690528fb77ce0bba2fa9f7>,
 <Future: pending, key: main-ba42d1f1d943df5c10c48f6a164083bd>,
 <Future: pending, key: main-6b8848d5419342c271de428b54412321>,
 <Future: pending, key: main-66258a637050c50dcab8b159f14d9ec7>,
 <Future: pending, key: main-6e57395ee0ce52a2b225f11107cd5b48>,
 <Future: pending, key: main-84a1d2e404f2e98dd2487f9277bd221c>,
 <Future: pending, key: main-0ca9727abd0c3adc7c9952d3754d09ee>,
 <Future: pending, key: main-a74ebe9b364d5b3cf384c23972c1fb35>,
 <Future: pending, key: main-6d83b46c350

In [ ]:
# Getting results
results = client.gather(futures)

In [ ]:
# Closing client
client.close()
cluster.close()

In [ ]:
# Saving results
torch.save(obj = results,
           f = "ablation_nystrom_ntrial={0}_n={1}_d={2}_noise={3}.pt".format(ntrial,n,d,noise))